### Import Modules, Change Directory, Set Variables

In [1]:
output_location = 'jupyter/jowilder_data' #CHANGE: output directory
output_name = '12182019 jowidler assessment 4.tsv' #CHANGE: output tsv file name
assessment_n = 4 # #CHANGE: assessment through which you want all features

In [2]:
import os
from zipfile import ZipFile
import pandas as pd
import json
if not os.path.split(os.getcwd())[-1] == 'opengamedata':
    os.chdir('..')
from game_info.Jowilder import Jowilder_Enumerators as je
os.getcwd()

'/Users/johnmccloskey/Development/opengamedata'

### Look to see what proc_zips are available, choose one to open

In [3]:
data_dir = 'data/JOWILDER'
files = [x for x in sorted(os.listdir(data_dir),reverse=True) if 'proc.zip' in x]
get_file_by_n = lambda n: os.path.join(data_dir, files[n])
[f'{i}: {f}' for i,f in enumerate(files)][:10]

['0: JOWILDER_20191218_to_20191218_ba39771_proc.zip',
 '1: JOWILDER_20191201_to_20191218_de09c18_proc.zip',
 '2: JOWILDER_20191201_to_20191217_6d66abe_proc.zip',
 '3: JOWILDER_20191201_to_20191213_0e39a60_proc.zip',
 '4: JOWILDER_20191201_to_20191211_dc8553c_proc.zip',
 '5: JOWILDER_20191201_to_20191210_d333c49_proc.zip',
 '6: JOWILDER_20191201_to_20191206_1bcf69b_proc.zip',
 '7: JOWILDER_20191201_to_20191206_160cf79_proc.zip',
 '8: JOWILDER_20191201_to_20191204_138f261_proc.zip',
 '9: JOWILDER_20191201_to_20191203_9700a60_proc.zip']

In [4]:
read_file = 1 # CHANGE: This is the enum of the file you wish to open

### Import to dataframe, verify roughly that is it what is expected

In [5]:
zf = ZipFile(get_file_by_n(read_file))
with zf.open(zf.namelist()[0]) as f:
    df = pd.read_csv(f,index_col='sessionID')
df.head()

,lvl0_EventCount,lvl1_EventCount,lvl2_EventCount,lvl3_EventCount,lvl4_EventCount,lvl5_EventCount,lvl6_EventCount,lvl7_EventCount,lvl8_EventCount,lvl9_EventCount,...,sess_avgTimePerWord,sess_avgTimePerTextBox,sess_stDevTimePerTextBox,sess_words_per_second,sess_count_notebook_uses,sess_avg_time_between_clicks,save_code,music,hq,fullscreen
sessionID,,,,,,,,,,,,,,,,,,,,,
19110000090781600,34,30.0,41.0,50.0,67.0,31.0,98.0,74.0,43.0,53.0,...,0,16.005742,0,4.125283,8,161.139635,0,1,0,0
19110007435189656,38,25.0,79.0,33.0,58.0,14.0,148.0,66.0,41.0,49.0,...,0,2.617716,0,11.738549,3,1.428273,0,1,0,0
19110008053486390,34,33.0,43.0,37.0,55.0,19.0,97.0,49.0,38.0,41.0,...,0,3.781058,0,8.453793,2,1.027874,0,1,0,0
19110008122038204,64,25.0,46.0,49.0,59.0,31.0,96.0,49.0,42.0,49.0,...,0,8.934659,0,3.476326,4,2.515577,0,1,0,0
19110008462915572,34,27.0,54.0,50.0,64.0,17.0,97.0,94.0,53.0,45.0,...,0,5.943833,0,5.567715,8,1.516161,0,1,0,0


In [6]:
df.shape

(3087, 389)

### Import Schema, look at available features

In [7]:
schema_path = 'schemas/JSON/JOWILDER.json'
with open(schema_path) as f:
    schema = json.load(f)
features = schema['features']
lvl_feature = lambda n,f: f'lvl{n}_{f}'
def get_custom_count_feature_list(f,to_assessment=None):
    prefix = features['per_custom_count'][f]['prefix']
    if to_assessment and prefix=='Q':
        max_n = je.assessment_to_last_q[to_assessment]+1
    else:
        max_n = features['per_custom_count'][f]['count']
    return [f'{prefix}{n}_{f}' for n in range(max_n)]

In [8]:
print(*[[k,list(v.keys())] for k,v in features.items()],sep='\n\n')

['perlevel', ['EventCount', 'count_clicks', 'count_hovers', 'avgTimePerTextBox', 'time_in_level', 'meaningful_action_count', 'words_per_second', 'count_notebook_uses', 'avg_time_between_clicks']]

['per_custom_count', ['num_guesses', 'answers', 'A1', 'A2', 'A3', 'A1_time', 'A2_time', 'A3_time', 'quiz_response']]

['aggregate', ['sessionID', 'persistentSessionID', 'max_level', 'sessDuration', 'sess_count_clicks', 'sess_count_hovers', 'sess_meaningful_action_count', 'sess_EventCount', 'sess_timeToRead', 'sess_avgTimePerWord', 'sess_avgTimePerTextBox', 'sess_stDevTimePerTextBox', 'sess_words_per_second', 'sess_count_notebook_uses', 'sess_avg_time_between_clicks', 'save_code', 'music', 'hq', 'fullscreen']]


### Drop columns past given assessment, drop rows that did not answer the last question

In [9]:
def features_through_assessment_n(n):
    ret = []
    for lvl in range(je.assessement_to_lvl[n]+1):
        for f in features['perlevel']:
            ret.append(lvl_feature(lvl,f))
    for f in features['per_custom_count']:
        ret.extend(get_custom_count_feature_list(f, to_assessment=n))
    ret.extend([f for f in features['aggregate'] if f != 'sessionID'])
    return ret
last_q_subset = [f'Q{je.assessment_to_last_q[assessment_n]}_A1'] if assessment_n is not 1 else \
    ['Q2_A1','Q3_A1']
assessment_df = df[features_through_assessment_n(assessment_n)]
assessment_df = assessment_df.dropna(subset=last_q_subset, how='all')

In [10]:
assessment_df.shape

(259, 380)

In [11]:
assessment_df.head()

,lvl0_EventCount,lvl0_count_clicks,lvl0_count_hovers,lvl0_avgTimePerTextBox,lvl0_time_in_level,lvl0_meaningful_action_count,lvl0_words_per_second,lvl0_count_notebook_uses,lvl0_avg_time_between_clicks,lvl1_EventCount,...,sess_avgTimePerWord,sess_avgTimePerTextBox,sess_stDevTimePerTextBox,sess_words_per_second,sess_count_notebook_uses,sess_avg_time_between_clicks,save_code,music,hq,fullscreen
sessionID,,,,,,,,,,,,,,,,,,,,,
19110000090781600,34,26.0,6.0,1.672167,63.535,3.0,3.970336,NaN,2.096680,30.0,...,0,16.005742,0,4.125283,8,161.139635,0,1,0,0
19110009114094480,21,18.0,1.0,1.839250,44.510,1.0,4.523686,NaN,1.195588,27.0,...,0,7.617183,0,4.731634,6,2.034186,donethework,1,0,0
19110012454548376,43,36.0,5.0,1.545500,73.820,4.0,6.184182,NaN,1.584343,114.0,...,0,7.371897,0,7.551905,11,1.610426,0,1,1,True
19110013234266620,2,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,...,0,4.979176,0,9.855149,6,1.309599,0,1,0,0
19110014392453668,2,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,...,0,6.194067,0,6.349095,1,1.576921,0,1,0,0


### Export

In [12]:
### Import to dataframe, verify roughly that is it what is expected
assessment_df.to_csv(os.path.join(output_location,output_name), sep='\t')